In [ ]:
!pip install trimesh rtree

import numpy as np
import trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 18.7 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Cargar el modelo STL (asegúrate de que 'modelo.stl' esté en el directorio o proporciona la ruta correcta)
mesh = trimesh.load('/content/drive/MyDrive/Colab Notebooks/Retos/Monostatic_Body_Release.STL')

# Calcular el centro de masa del modelo
centro_masa = mesh.center_mass

def vector_desde_centro(theta, phi):
    """
    Dada una dirección definida por theta y phi (en radianes),
    retorna el vector desde el centro de masa del modelo hasta
    el punto en la superficie donde intersecta el rayo.

    Parámetros:
      theta : ángulo polar (medido desde el eje z positivo)
      phi   : ángulo azimutal (en el plano xy, medido desde el eje x positivo)

    Retorna:
      vector : np.array de tamaño (3,) que va desde el centro de masa
               hasta el punto de intersección en la superficie.
    """
    # Convertir coordenadas esféricas a vector unitario en coordenadas cartesianas
    # Asumimos:
    #   x = sin(theta)*cos(phi)
    #   y = sin(theta)*sin(phi)
    #   z = cos(theta)
    direccion = np.array([
        np.sin(theta) * np.cos(phi),
        np.sin(theta) * np.sin(phi),
        np.cos(theta)
    ])

    # Lanzar un rayo desde el centro de masa en la dirección calculada
    # La función ray.intersects_location retorna:
    #  - locations: puntos de intersección
    #  - index_ray: índice del rayo (en este caso siempre 0)
    #  - index_tri: índice del triángulo intersectado
    locations, index_ray, index_tri = mesh.ray.intersects_location(
        ray_origins=[centro_masa],
        ray_directions=[direccion]
    )

    # Verificar si se encontró alguna intersección
    if len(locations) == 0:
        raise ValueError("No se encontró intersección con la superficie para la dirección dada.")

    # Si hay múltiples intersecciones, tomar la más cercana (distancia mínima al centro de masa)
    distancias = np.linalg.norm(locations - centro_masa, axis=1)
    indice_min = np.argmin(distancias)
    punto_interseccion = locations[indice_min]

    # Retornar el vector desde el centro de masa hasta el punto de intersección
    return punto_interseccion - centro_masa

# Ejemplo de uso:
if __name__ == "__main__":
    # Ejemplo: theta = 45° (pi/4 radianes), phi = 60° (pi/3 radianes)
    theta = np.pi / 4
    phi = np.pi / 3

    try:
        vector_resultado = vector_desde_centro(theta, phi)
        print("El vector desde el centro de masa hasta la superficie es:", vector_resultado)
    except ValueError as e:
        print(e)


El vector desde el centro de masa hasta la superficie es: [ 9.25223831 16.02534684 18.50447663]


In [ ]:
# Calcular el centro de masa del modelo
centro_masa = mesh.center_mass

def normal_desde_superficie(theta, phi):
    """
    Dada una dirección definida por theta y phi (en radianes),
    lanza un rayo desde el centro de masa en dicha dirección y retorna
    el vector normal (unitario) a la superficie en el punto de intersección.

    Parámetros:
      theta : ángulo polar (medido desde el eje z positivo)
      phi   : ángulo azimutal (en el plano xy, medido desde el eje x positivo)

    Retorna:
      normal : np.array de tamaño (3,) que es el vector normal unitario a la superficie.
    """
    # Convertir coordenadas esféricas a un vector unitario en coordenadas cartesianas
    direccion = np.array([
        np.sin(theta) * np.cos(phi),
        np.sin(theta) * np.sin(phi),
        np.cos(theta)
    ])

    # Lanzar un rayo desde el centro de masa en la dirección calculada
    locations, index_ray, index_tri = mesh.ray.intersects_location(
        ray_origins=[centro_masa],
        ray_directions=[direccion]
    )

    # Verificar si se encontró alguna intersección
    if len(locations) == 0:
        raise ValueError("No se encontró intersección con la superficie para la dirección dada.")

    # Si hay múltiples intersecciones, se selecciona la más cercana al centro de masa
    distancias = np.linalg.norm(locations - centro_masa, axis=1)
    indice_min = np.argmin(distancias)

    # Obtener el índice del triángulo que se intersecta
    tri_idx = index_tri[indice_min]

    # Obtener la normal del triángulo (la cual usualmente ya viene normalizada)
    normal = mesh.face_normals[tri_idx]
    # Asegurarse de que el vector esté normalizado
    normal = normal / np.linalg.norm(normal)

    return normal

# Ejemplo de uso:
if __name__ == "__main__":
    # Ejemplo: theta = 45° (pi/4 radianes), phi = 60° (pi/3 radianes)
    theta = np.pi / 4
    phi = np.pi / 3

    try:
        normal_vector = normal_desde_superficie(theta, phi)
        print("El vector normal a la superficie es:", normal_vector)
        print("Magnitud:", np.linalg.norm(normal_vector))
    except ValueError as e:
        print(e)


El vector normal a la superficie es: [0.32518751 0.54714389 0.77128895]
Magnitud: 1.0
